<a href="https://colab.research.google.com/github/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/Topic_Modeling_with_BERTopic_Reclame_aqui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Topic Modeling with BERTopic - Reclame Aqui**

BERTopic is a topic modeling technique that leverages 🤗 transformers and a custom class-based TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions (https://maartengr.github.io/BERTopic/index.html).

## **Import dependecies**

In [139]:
import pandas as pd # for data manipulation
import os # for interacting with the operating system
import nltk # for natural language processing
import string # for string manipulation
import re # for for regular expressions
try:
  from bertopic import BERTopic # for topic modeling
  from spellchecker import SpellChecker # for spell check
except:
  !pip install bertopic
  !pip install pyspellchecker
  os.kill(os.getpid(), 9)

In [140]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [141]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [142]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [143]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

### **Load data from [Github](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui.git)**

In [144]:
!git clone https://github.com/punkmic/Topic-Modeling-Reclame-Aqui.git

fatal: destination path 'Topic-Modeling-Reclame-Aqui' already exists and is not an empty directory.


### **Run this cell to update files from remote repository**

In [145]:
# Change directory
%cd /content/Topic-Modeling-Reclame-Aqui 

# Update files from remote repository
!git pull 

# Return to work directory
%cd ..

# Check current directory
!pwd

/content/Topic-Modeling-Reclame-Aqui
Already up to date.
/content
/content


### **Read csv file using read_csv() method as Dataframe**

In [178]:
# Set the path to data
path_csv = "/content/Topic-Modeling-Reclame-Aqui/corpus.csv"

# Read the CSV file using the read_csv method
df = pd.read_csv(path_csv)

# Print the first 5 rows of the DataFrame
df.head(5)

,title,text
0,Pedido Cancelado sem justificativa após uma se...,Eu estava pesquisando bastante uma nova TV par...
1,Pedido cancelado,Eu sinceramente estou decepcionada com o Amazo...
2,Cobrança indevida,Cancelei meu plano antes de terminar o período...
3,PEDIDO REINCIDENTE,"OLHA FIZ COMPRA VEIO ERRADA, E VEIO ERRADO NOV..."
4,Assinatura para vender na amazon Brasil,Eu me inscrevi na Amazon para realizar vendas ...


This dataset contains just two columns called title and text 

In [179]:
print(df.shape)

(12760, 2)


There are 12760 rows in this dataset. To make sure that we have unique rows let's remove duplicate rows.

In [180]:
print(f"Shape before remove duplicates: {df.shape}")

# User the drop_duplicated method to drop duplicates rows
df = df.drop_duplicates(subset="text")

print(f"Shape after remove duplicates: {df.shape}")

Shape before remove duplicates: (12760, 2)
Shape after remove duplicates: (10510, 2)


### **Now let's lower each element in the Dataframe, remove unwanted text and join the two columns**

In [181]:
# apply the str.lower() method to each element in the dataframe
df = df.applymap(str.lower)

# Use the replace() method to replace the string with an empty string
df = df.replace(re.compile('\[editado pelo reclame aqui\]|editado pelo reclame aqui|Editado pelo Reclame Aqui'), '')

# join columns
df["documents"] = df["title"] + " " + df["text"]
df.head()

,title,text,documents
0,pedido cancelado sem justificativa após uma se...,eu estava pesquisando bastante uma nova tv par...,pedido cancelado sem justificativa após uma se...
1,pedido cancelado,eu sinceramente estou decepcionada com o amazo...,pedido cancelado eu sinceramente estou decepc...
2,cobrança indevida,cancelei meu plano antes de terminar o período...,cobrança indevida cancelei meu plano antes de ...
3,pedido reincidente,"olha fiz compra veio errada, e veio errado nov...",pedido reincidente olha fiz compra veio errada...
4,assinatura para vender na amazon brasil,eu me inscrevi na amazon para realizar vendas ...,assinatura para vender na amazon brasil eu me ...


In [149]:
# Drop the old index column
df.reset_index(inplace = True, drop = True)

# Print the shape
df.shape

(10510, 3)

## **Preprocessing**

### **Tokenization**

Tokenization aims to breaking text down into its component parts

In [150]:
WORD_TOKENIZER = nltk.tokenize.word_tokenize
def tokenize(text, lowercase=True):
  if lowercase:
    text = text.lower()
  return WORD_TOKENIZER(text, language="portuguese")

### **Stem** 

Stem the tokens. This step aims to remove morphological affixes and normalize to standardized stem forms

In [151]:
STEMMER = nltk.PorterStemmer()
def stem(tokens):
  return [STEMMER.stem(token) for token in tokens]

### **Lemmatize**

Lemmatize the tokens. Retains more natural forms than stemming, but assumes all tokens nons unless tokens are passed as (word, pos) tuples.

In [152]:
LEMMATIZER = nltk.WordNetLemmatizer()
def lemmatize(tokens):
  lemmas = []
  for token in tokens:
    if isinstance(token, str):
      # For str token
      lemmas.append(LEMMATIZER.lemmatize(token)) 
    else:
      # For tuple of (str, pos)
      lemmas.append(LEMMATIZER.lemmatize(*tokens)) 
  return lemmas

### **Remove stopwords**

Stop words are things like articles and conjunctions that usually do not offer a lot of value in an analysis.

In [153]:
def remove_stopwords(tokens, stopwords=None):

  # Use the default stop words if none is passed
  if stopwords is None:
    stopwords = nltk.corpus.stopwords.words("portuguese")
  
  # Filter the list of tokens to exclude the stop word tokens
  return [token for token in tokens if token not in stopwords]

### **Remove hyperlinks**

Removes http/s links from the tokens.

In [154]:
def remove_links(tokens):
  # Filter tokens that starts with "http://" or "https://"
  return [token for token in tokens 
          if not token.startswith("http://")
          and not token.startswith("https://")]

### **Remove numbers**

In [155]:
def remove_numbers(tokens):
  # Filter number tokens
  return [token for token in tokens if not token.isdigit()]

In [156]:
def remove_date(tokens):
  # Compile a regular expression to match dates in the format dd/mm or dd/mm/yyyy
  date_regex = re.compile(r'\d{2}/\d{2}(/\d{4})?')

  # Use the regex to find all the tokens that match the date pattern
  dates = [token for token in tokens if date_regex.fullmatch(token)]

  # Filter the list of tokens to exclude the date tokens
  filtered_tokens = [token for token in tokens if token not in dates]

  # Return the filtered tokens
  return filtered_tokens

### **Remove punctuation**

In [157]:
def remove_punctuation(tokens,
                       strip_mentions=False,
                       strip_hashtags=False,
                       strict=False):

    # Use the sub method to remove all punctuation characters
    tokens = [re.sub(r"[,!?.]", "", t) for t in tokens ] 

    # Remove punctuation
    #tokens = [t for t in tokens if t not in string.punctuation]

    # Remove @ symbol from left side of tokens
    if strip_mentions:
        tokens = [t.lstrip('@') for t in tokens]

    # Remove # symbol from left side of tokens
    if strip_hashtags:
        tokens = [t.lstrip('#') for t in tokens]

    return tokens

### **Remove short tokens**

In [158]:
def remove_short_tokens(tokens):
  # Filter the list of tokens to exclude tokens that are shorter than four letters
  filtered_tokens = [token for token in tokens if len(token) >= 4]

  # Return the filtered tokens
  return filtered_tokens

### **Correction of spelling errors and abbreviations**

In [159]:
def check_spell_errors(text):
  # Create a SpellChecker object
  spell = SpellChecker(language='pt')

  # Correct the spelling errors in the text
  corrected_text = spell.correction(text)

  # If no correction is present user the original text
  if corrected_text == None:
     corrected_text =  text
  
  # Return the corrected text
  return corrected_text

In [1]:
def preprocessing(documents):
  corpus = []

  # process each document and append to corpus list
  for i, text in enumerate(documents):
    if i % 1000 == 0:
      print(f"Processed {i} documents")
    text = check_spell_errors(text)
    tokens = tokenize(text)
    tokens = remove_links(tokens)
    tokens = remove_punctuation(tokens, strip_mentions=True, strip_hashtags=True)
    tokens = remove_numbers(tokens)
    tokens = remove_date(tokens)
    tokens = remove_short_tokens(tokens)
    tokens = remove_stopwords(tokens)
    tokens = lemmatize(tokens) 
    corpus.append(' '.join(tokens))
  return corpus

In [185]:
# Print the first document before and after pre-processing it
corpus = preprocessing(df.documents) 
print(df.documents[0])
print()
corpus[0]

pedido cancelado sem justificativa após uma semana da compra eu estava pesquisando bastante uma nova tv para comprar e resolvi aguardar a semana do cliente, porque como ocorreu tiveram vários descontos e promoções nessa semana, sendo assim recebi descontos, cashback e cupons de varias plataformas e assim decidi efetuar a compra aquela que eu entendesse ser o melhor custo beneficio. sendo assim no dia 12 de setembro de 2022 recebi uma oferta de produto da amazon que entendi estar com um preço muito bom além de ter cachback e efetuei a compra que já aguardei até a semana do consumidor para efetuar com toda a expectativa. como estavamos anciosos e mesmo estando dentro do prazo de entrega hoje no dia 19/09 resolvi enviar uma mensagem perguntando quando o pedido seria enviado, ja que após a confirmação da compra e pagamento, passado 7 dias não tive nenhum retorno, já que complei inclusive na amazon, um site em que confio e tambem sou assinante. para a minha surpresa após o pedido de informa

'pedido cancelado justificativa após semana compra pesquisando bastante nova comprar resolvi aguardar semana cliente porque ocorreu vários descontos promoções nessa semana sendo assim recebi descontos cashback cupons varias plataformas assim decidi efetuar compra entendesse melhor custo beneficio sendo assim setembro recebi oferta produto amazon entendi preço além cachback efetuei compra aguardei semana consumidor efetuar toda expectativa estavamos anciosos estando dentro prazo entrega hoje resolvi enviar mensagem perguntando pedido enviado após confirmação compra pagamento passado dia nenhum retorno complei inclusive amazon site confio tambem assinante surpresa após pedido informação pedido retorno email estaria sendo cancelado sendo nenhuma justificativa após dia realização aguardei chegar periodo compro ansiosamente aproveitar descontos produto familia aguardava além cancelar compra produto justificativa empresa tirou toda oportunidades descontos efetuar compra semana consumidor lev

## **Training a BERTopic Model**

As our data language is portuguese we will going to set language to multilingual.

### **Enabling the GPU**

We will use the GPU provided by COLAB to accelarate our model training. To enable GPUs for the notebook:
1- Navigate to Edit -> Notebook Settings
2- Select GPU from the Hardware Accelerator drop-down

In [162]:
# verify if GPU is selected
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Dec 10 22:25:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    34W /  70W |   1386MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [184]:
# Create a new BERTopic model using multilingual option
topic_model = BERTopic(language="multilingual", calculate_probabilities=True, verbose=True)

# Train model 
topics, probs = topic_model.fit_transform(corpus)

Batches:   0%|          | 0/329 [00:00<?, ?it/s]

2022-12-10 22:57:38,819 - BERTopic - Transformed documents to Embeddings
2022-12-10 22:57:51,367 - BERTopic - Reduced dimensionality
2022-12-10 22:57:54,626 - BERTopic - Clustered reduced embeddings


## **Extracting Topics**

In [164]:
# print the most frequent topics
freq = topic_model.get_topic_info()
freq.head(5)

,Topic,Count,Name
0,-1,4481,-1_produto_compra_pedido_entrega
1,0,1449,0_amazon_prime_assinatura_cartão
2,1,568,1_entrega_atraso_prazo_data
3,2,506,2_casas_bahia_entrega_transportadora
4,3,332,3_cancelamento_cancelar_pedido_cancelado


The table above shows the five most freqeuente topics and the words present on it extract by BERTopic. -1 refers to all outliers and should be ignored.

In [165]:
# show the most frequent topic
topic_model.get_topic(1)

[('entrega', 0.038574590353602285),
 ('atraso', 0.03138799047938097),
 ('prazo', 0.026177832530564146),
 ('data', 0.021252743617463054),
 ('atrasada', 0.019350480528862284),
 ('pedido', 0.016015215446197228),
 ('previsão', 0.015551358578147887),
 ('dia', 0.01432637353225581),
 ('ainda', 0.013714904194269012),
 ('hoje', 0.013035796523943613)]

**Note:** BERTopic is stocastich which means that the topics might differ across runs this is mostly due to the stocastisch nature of UMAP

## **Visualization**

### **Intertopic Distance Map**

This graph shows the distance intertopic and help us understand the promixity of topics

In [166]:
topic_model.visualize_topics()

### **Visualize Topic Hierarchy**

The topics that were created can be hierarchically reduced. This visualization shows how the topics relate to one another.

In [167]:
topic_model.visualize_hierarchy(top_n_topics=50)

### **Visualize Terms**

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation.

In [168]:
topic_model.visualize_barchart(top_n_topics=8)

### **Visualize Topic Similarity**

This plot shows a similarity matrix by simply applying cosine similarities through those topic embeddings generate by BERTopic through both c-TF-IDF and embeddings. This matrix indicate how similar certain topics are to each other.

In [169]:
topic_model.visualize_heatmap(n_clusters=20, width=1000, height=1000)

Visualize Term Score Decline

Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added.

In [170]:
topic_model.visualize_term_rank()

### **Term search**

In [171]:
similar_topics, similarity = topic_model.find_topics("blackfriday", top_n=5)
similar_topics

[53, 34, 58, 51, 57]

In [172]:
topic_model.get_topic(54)

[('vestido', 0.16442820899008487),
 ('vestidos', 0.10570586280990027),
 ('casamento', 0.07330655331830517),
 ('shein', 0.04845673044271494),
 ('roupa', 0.04447837028496432),
 ('novembro', 0.04362532144714115),
 ('previsto', 0.035093701289421776),
 ('acessórios', 0.03313895805006228),
 ('fiscalização', 0.0327058936361859),
 ('curitiba', 0.03249433646812514)]

### **Topic Reduction**

In [173]:
#new_topic_model = topic_model.reduce_topics(df.documents, nr_topics=40)
#new_topics = new_topic_model.topics_
#new_probs = new_topic_model.probabilities_

In [174]:
#new_topic_model.visualize_topics()